# The required imports

In [1]:
# we either import locally (after the repo was cloned)
# or we first clone it and then use it (e.g., when run in Google Colab)
shell = get_ipython().__class__.__name__ 

if shell == 'Shell':
    # imports when launched in e.g., Google Colab
    !git clone https://github.com/filipzz/athena.git r2b2
    from r2b2.code.athena.athena import AthenaAudit
    from r2b2.code.athena.contest import Contest
    from r2b2.code.athena.audit import Audit
else: # shell ==  'ZMQInteractiveShell' or shell == 'TerminalInteractiveShell'
    # local imports if you run it with e.g., Jupyter
    from athena.athena import AthenaAudit
    from athena.contest import Contest
    from athena.audit import Audit

import pandas as pd

# Setting risk limit

In [2]:
# Set risk limit of the audit
risk_limit = 0.1 

# Loading data from a file

In [3]:
# address of the file with election results the file can be either
# an url:
#election_results = "https://raw.githubusercontent.com/gwexploratoryaudits/brla_explore/poorvi/R2Audits/Athena/Scripts/2020MontgomeryPrimary/2020_montgomery_formatted.json"
# or a local file:
#election_results = "athena/test_data/simple.json"
#contest_name = "contest_1"
#another example
election_results = "athena/test_data/2020_montgomery_formatted.json"
contest_name = "d_president"

# Check if the load is successful

In [4]:

w = Audit("athena", risk_limit)

w.read_election_results(election_results)

In [5]:
print(w.observations)

{'d_president': [[], [], [], [], [], [], [], [], [], [], []], 'd_congress': [[], []], 'd_senator': [[], []], 'd_cc_1_2_2021': [[], []], 'd_cc_1_3_2021': [[], []], 'r_10th': [[], [], []], 'r_senator': [[], [], []], 'r_42nd': [[], []], 'r_cc_1_2_2021': [[], []]}


In [6]:
w.load_contest(contest_name)
w.show_election_results()

,Candidates,Results
0,Bennet,49
1,Biden,28135
2,Bloomberg,682
3,Buttigieg,509
4,Gabbard,130
5,Klobuchar,396
6,Patrick,23
7,Sanders,5466
8,Steyer,57
9,Warren,1074


# Example - running an audit

In [7]:
# create an Audit object with risk_limit
w = Audit("athena", risk_limit)

# read data from a file
w.read_election_results(election_results)

# load selected contest
w.load_contest(contest_name)

# predict round sizes
print(w.predict_round_sizes([.7, .8, .9]))

# set observations: 
# * number of total ballots drawn in the round
# * number of relevant ballots for the contest
# * a list of votes sampled for each candidate
#w.set_observations(50, 19, [0, 11, 0, 1, 0, 0, 0, 6, 1, 0, 0])
#print(w.predict_round_sizes([.7, .8, .9]))
# results for the second round:
#w.set_observations(29, 10, [0, 6, 0, 0, 0, 0, 0, 4, 0, 0, 0])
#w.present_state()


[[0.7, 21], [0.8, 25], [0.9, 29]]


# More examples

You may compare with data from: [https://github.com/gwexploratoryaudits/brla_explore/blob/poorvi/R2Audits/Athena/Scripts/2020MontgomeryPrimary/2020_montgomery_pvalue_multiple_round_tests.json](https://github.com/gwexploratoryaudits/brla_explore/blob/poorvi/R2Audits/Athena/Scripts/2020MontgomeryPrimary/2020_montgomery_pvalue_multiple_round_tests.json)

In [8]:

# create an Audit object with risk_limit
w = Audit("athena", risk_limit)

# read data from a file
w.read_election_results(election_results)

# load selected contest
w.load_contest("d_president")

# test 1 (d_president) 
#w.load_contest("d_president")
w.set_observations(50, 19, [0, 11, 1, 0, 0, 0, 0, 6, 1, 0, 0])
w.set_observations(29, 10, [0, 6, 0, 0, 0, 0, 0, 4, 0, 0, 0])
w.set_observations(29, 10, [1, 1, 1, 1, 1, 1, 0, 4, 0, 0, 0])
w.present_state()
# test 2 - (d_president): 
#w.load_contest("d_president")
#w.set_observations(16, 16, [0, 10, 1, 0, 0, 0, 0, 4, 1, 0, 0])
#w.set_observations(29, 29, [1, 10, 0, 1, 1, 1, 1, 13, 0, 1, 0])
# test 3 (d_pres)
#w.load_contest("d_president")
#w.set_observations(22, 22, [0, 11, 1, 0, 0, 0, 0, 9, 1, 0, 0])
#w.set_observations(7, 7, [0, 6, 0, 0, 0, 0, 0, 1, 0, 0, 0])
# test 4 - congress - ok
#w.load_contest("d_congress")
#w.set_observations(42, 42, [17, 25])
# test 5 - ok
#w.load_contest("d_congress")
#w.set_observations(27, 27, [10, 17])
#w.set_observations(27, 27, [10, 17])
# test 6 
#w.load_contest("d_congress")
#w.set_observations(31, 31, [12, 19])
#w.set_observations(31, 31, [12, 19])
# test7 
#w.load_contest("d_senator")
#w.set_observations(17, 17, [11, 6])
#w.set_observations(9, 9, [5, 4])
#test8 - the same problem
#w.load_contest("d_senator")
#w.set_observations(12, 12, [8, 4])
#w.set_observations(12, 12, [8, 4])
#test 9
#w.load_contest("d_senator")
#w.set_observations(40, 40, [27, 13])
#w.set_observations(40, 40, [27, 13])

#w.present_state()



,Candidates,Results,Round 1,Round 2,Round 3,Total,Required
0,Bennet,49,0,0,1,1,
1,Biden,28135,11,6,1,18,24
2,Bloomberg,682,1,0,1,2,
3,Buttigieg,509,0,0,1,1,
4,Gabbard,130,0,0,1,1,
5,Klobuchar,396,0,0,1,1,
6,Patrick,23,0,0,0,0,
7,Sanders,5466,6,4,4,14,
8,Steyer,57,1,0,0,1,
9,Warren,1074,0,0,0,0,


In [9]:
w.load_contest("d_senator")
w.set_observations(40, 40, [27, 13])
w.set_observations(40, 40, [27, 13])

w.present_state()


,Candidates,Results,Round 1,Round 2,Total,Required
0,Fogel,16339,27,27,54,55
1,Griggs,3514,13,13,26,
2,,Sum,40,40,,
3,,LR,0.9565,0.9148,,
4,,P-Value,0.0194,0.0359,,


In [10]:
print(w.predict_round_sizes([.7, .8, .9]))


[[0.7, 203], [0.8, 223], [0.9, 247]]


In [12]:
w

audit type: athena
alpha: 0.1
observations: [[27, 54], [13, 26]]
status: {'d_president': {"round_number": 4, "min_kmins": [0, 24, 0, 0, 0, 0, 0, 0, 0, 0, 0], "risks": [0.1683235777259096, 1.6863216195463144, 8.505545796429123]}, 'd_congress': {"round_number": 1, "min_kmins": [], "risks": []}, 'd_senator': {"round_number": 3, "min_kmins": [55, 0], "risks": [0.019368990308389875, 0.03590104177511345]}, 'd_cc_1_2_2021': {"round_number": 1, "min_kmins": [], "risks": []}, 'd_cc_1_3_2021': {"round_number": 1, "min_kmins": [], "risks": []}, 'r_10th': {"round_number": 1, "min_kmins": [], "risks": []}, 'r_senator': {"round_number": 1, "min_kmins": [], "risks": []}, 'r_42nd': {"round_number": 1, "min_kmins": [], "risks": []}, 'r_cc_1_2_2021': {"round_number": 1, "min_kmins": [], "risks": []}}

# BRAVO vs Athena vs Minerva

## Minerva 

* passes for 57 out of 100
* cares only for p-value

In [13]:
# Minerva would pass the test for kmin = 57
wmin = Audit("athena", risk_limit)
wmin.read_election_results(election_results)
wmin.load_contest("d_president")
# minerva
wmin.set_observations(100, 100, [0, 57, 0, 0, 0, 0, 0, 43, 0, 0, 0])

In [14]:
wmin.present_state()


,Candidates,Results,Round 1,Total,Required
0,Bennet,49,0,0,
1,Biden,28135,57,57,69
2,Bloomberg,682,0,0,
3,Buttigieg,509,0,0,
4,Gabbard,130,0,0,
5,Klobuchar,396,0,0,
6,Patrick,23,0,0,
7,Sanders,5466,43,43,
8,Steyer,57,0,0,
9,Warren,1074,0,0,


## Athena

* passes for 69 out of 100
* cares for both:
    * p-value (to be below .1)
    * LR (to be obove 1)
   $\sum_{k=1}^{\infty}d_v$

In [15]:
# athena
wath = Audit("athena", risk_limit)
wath.read_election_results(election_results)
wath.load_contest("d_president")
wath.set_observations(100, 100, [0, 68, 0, 0, 0, 0, 0, 32, 0, 0, 0])

In [16]:
wath.present_state()

,Candidates,Results,Round 1,Total,Required
0,Bennet,49,0,0,
1,Biden,28135,68,68,69
2,Bloomberg,682,0,0,
3,Buttigieg,509,0,0,
4,Gabbard,130,0,0,
5,Klobuchar,396,0,0,
6,Patrick,23,0,0,
7,Sanders,5466,32,32,
8,Steyer,57,0,0,
9,Warren,1074,0,0,


## Bravo

* passes for 70 out of 100
* cares for LR to be over 10

In [17]:
w = Audit("athena", risk_limit)
w.read_election_results(election_results)
w.load_contest("d_president")
w.set_observations(100, 100, [0, 70, 0, 0, 0, 0, 0, 30, 0, 0, 0])


In [18]:
w.present_state()

,Candidates,Results,Round 1,Total,Required
0,Bennet,49,0,0,
1,Biden,28135,70,70,69
2,Bloomberg,682,0,0,
3,Buttigieg,509,0,0,
4,Gabbard,130,0,0,
5,Klobuchar,396,0,0,
6,Patrick,23,0,0,
7,Sanders,5466,30,30,
8,Steyer,57,0,0,
9,Warren,1074,0,0,
